In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Read the raw data

In [4]:
df = pd.read_csv("/kaggle/input/uber-uncleaned-dataset/Uber Uncleaned Dataset")
df.head(7)

,id,timestamp,timezone,source,destination,cab_type,product_id,name,price,distance,...,precipIntensityMax,uvIndexTime,temperatureMin,temperatureMinTime,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime
0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,1.544953e+09,America/New_York,Haymarket Square,North Station,Lyft,lyft_line,Shared,5.0,0.44,...,0.1276,1544979600,39.89,1545012000,43.68,1544968800,33.73,1545012000,38.07,1544958000
1,4bd23055-6827-41c6-b23b-3c491f24e74d,1.543284e+09,America/New_York,Haymarket Square,North Station,Lyft,lyft_premier,Lux,11.0,0.44,...,0.1300,1543251600,40.49,1543233600,47.30,1543251600,36.20,1543291200,43.92,1543251600
2,981a3613-77af-4620-a42a-0c0866077d1e,1.543367e+09,America/New_York,Haymarket Square,North Station,Lyft,lyft,Lyft,7.0,0.44,...,0.1064,1543338000,35.36,1543377600,47.55,1543320000,31.04,1543377600,44.12,1543320000
3,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,1.543554e+09,America/New_York,Haymarket Square,North Station,Lyft,lyft_luxsuv,Lux Black XL,26.0,0.44,...,0.0000,1543507200,34.67,1543550400,45.03,1543510800,30.30,1543550400,38.53,1543510800
4,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,1.543463e+09,America/New_York,Haymarket Square,North Station,Lyft,lyft_plus,Lyft XL,9.0,0.44,...,0.0001,1543420800,33.10,1543402800,42.18,1543420800,29.11,1543392000,35.75,1543420800
5,f6f6d7e4-3e18-4922-a5f5-181cdd3fa6f2,1.545071e+09,America/New_York,Haymarket Square,North Station,Lyft,lyft_lux,Lux Black,16.5,0.44,...,0.0221,1545066000,34.19,1545048000,40.66,1545022800,27.39,1545044400,34.97,1545080400
6,462816a3-820d-408b-8549-0b39e82f65ac,1.543209e+09,America/New_York,Back Bay,Northeastern University,Lyft,lyft_plus,Lyft XL,10.5,1.08,...,0.1245,1543251600,40.67,1543233600,46.46,1543255200,37.45,1543291200,43.81,1543251600


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693071 entries, 0 to 693070
Data columns (total 53 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           693071 non-null  object 
 1   timestamp                    693071 non-null  float64
 2   timezone                     693071 non-null  object 
 3   source                       693071 non-null  object 
 4   destination                  693071 non-null  object 
 5   cab_type                     693071 non-null  object 
 6   product_id                   693071 non-null  object 
 7   name                         693071 non-null  object 
 8   price                        637976 non-null  float64
 9   distance                     686140 non-null  float64
 10  surge_multiplier             693071 non-null  object 
 11  latitude                     693071 non-null  float64
 12  longitude                    693071 non-null  float64
 13 

In [6]:
df.describe()

,timestamp,price,distance,latitude,longitude,temperature,apparentTemperature,precipIntensity,precipProbability,windSpeed,...,precipIntensityMax,uvIndexTime,temperatureMin,temperatureMinTime,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime
count,6.930710e+05,637976.000000,686140.000000,693071.000000,693071.000000,678101.000000,693071.000000,693071.000000,693071.000000,693071.000000,...,693071.000000,6.930710e+05,693071.000000,6.930710e+05,693071.000000,6.930710e+05,693071.000000,6.930710e+05,693071.000000,6.930710e+05
mean,1.544046e+09,24.820782,3.284540,42.338172,-71.066151,41.334413,35.881222,0.008922,0.157648,6.318610,...,0.037374,1.544044e+09,33.457774,1.544042e+09,45.261313,1.544047e+09,29.731002,1.544048e+09,41.997343,1.544048e+09
std,6.891925e+05,133.627825,17.493832,0.047840,0.020302,13.752585,7.918707,0.026901,0.350456,3.806877,...,0.055214,6.912028e+05,6.467224,6.901954e+05,5.645046,6.901353e+05,7.110494,6.871862e+05,6.936841,6.910777e+05
min,1.543204e+09,2.500000,0.020000,42.214800,-71.105400,18.910000,12.130000,0.000000,0.000000,0.450000,...,0.000000,1.543162e+09,15.630000,1.543122e+09,33.510000,1.543154e+09,11.810000,1.543136e+09,28.950000,1.543187e+09
25%,1.543444e+09,9.000000,1.280000,42.350300,-71.081000,36.530000,31.910000,0.000000,0.000000,3.410000,...,0.000000,1.543421e+09,30.170000,1.543399e+09,42.570000,1.543439e+09,27.760000,1.543399e+09,36.570000,1.543439e+09
50%,1.543737e+09,13.500000,2.170000,42.351900,-71.063100,40.680000,35.900000,0.000000,0.000000,5.920000,...,0.000400,1.543770e+09,34.240000,1.543727e+09,44.680000,1.543788e+09,30.130000,1.543745e+09,40.950000,1.543788e+09
75%,1.544828e+09,22.500000,2.940000,42.364700,-71.054200,43.910000,40.080000,0.000000,0.000000,8.410000,...,0.091600,1.544807e+09,38.880000,1.544789e+09,46.910000,1.544814e+09,35.710000,1.544789e+09,44.120000,1.544818e+09
max,1.545161e+09,6500.000000,746.000000,42.366100,-71.033000,171.660000,57.220000,0.144700,1.500000,75.000000,...,0.145900,1.545152e+09,43.100000,1.545192e+09,57.870000,1.545109e+09,40.050000,1.545134e+09,57.200000,1.545109e+09


In [7]:
df.isnull().sum()

id                                 0
timestamp                          0
timezone                           0
source                             0
destination                        0
cab_type                           0
product_id                         0
name                               0
price                          55095
distance                        6931
surge_multiplier                   0
latitude                           0
longitude                          0
temperature                    14970
apparentTemperature                0
short_summary                      0
long_summary                       0
precipIntensity                    0
precipProbability                  0
humidity                        4678
windSpeed                          0
windGust                           0
windGustTime                       0
visibility                         0
temperatureHigh                    0
temperatureHighTime                0
temperatureLow                     0
t

In [33]:
df["destination"].unique()

array(['North Station', 'Northeastern University', 'West End',
       'Haymarket Square', 'South Station', 'Fenway', 'Theatre District',
       'Beacon Hill', 'Back Bay', 'North End', 'Financial District',
       'Boston University'], dtype=object)

### The price column has the highest number of missing values, approximately 8% of the dataset. Since this is our target variable, it's crucial to investigate the nature of these missing entries before deciding how to handle them.
### Let’s display only the rows where the price is missing to better understand any patterns or correlations in the missing data.

In [12]:
df[df['price'].isna()]

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,id,timestamp,timezone,source,destination,cab_type,product_id,name,price,distance,...,precipIntensityMax,uvIndexTime,temperatureMin,temperatureMinTime,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime
18,fa5fb705-03a0-4eb9-82d9-7fe80872f754,1.543674e+09,America/New_York,North End,West End,Uber,8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a,Taxi,NaN,1.11,...,0.0000,1543683600,31.71,1543658400,44.66,1543690800,28.06,1543658400,43.99,1543690800
31,eee70d94-6706-4b95-a8ce-0e34f0fa8f37,1.543795e+09,America/New_York,Beacon Hill,South Station,Uber,8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a,Taxi,NaN,2.48,...,0.0904,1543770000,36.45,1543726800,50.73,1543788000,35.85,1543748400,50.06,1543788000
40,7f47ff53-7cf2-4a6a-8049-83c90e042593,1.543524e+09,America/New_York,North Station,Fenway,Uber,8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a,Taxi,NaN,2.94,...,0.0000,1543510800,35.15,1543550400,44.66,1543510800,30.98,1543550400,38.30,1543510800
60,43abdbe4-ab9e-4f39-afdc-31cfa375dc25,1.544732e+09,America/New_York,North End,West End,Uber,8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a,Taxi,NaN,1.16,...,0.0001,1544716800,18.29,1544688000,33.83,1544731200,13.79,1544688000,32.85,1544734800
69,80db1c49-9d51-4575-a4f4-1ec23b4d3e31,1.543583e+09,America/New_York,North End,Beacon Hill,Uber,8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a,Taxi,NaN,2.67,...,0.0003,1543593600,28.79,1543579200,42.52,1543600800,26.41,1543575600,40.53,1543611600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693025,70ad9983-92dd-4cdf-af16-732e7e9ee9d0,1.544887e+09,America/New_York,South Station,Beacon Hill,Uber,8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a,Taxi,NaN,2.50,...,0.0081,1544893200,39.68,1544929200,54.46,1544896800,36.43,1544929200,53.79,1544896800
693047,84e05ef0-781a-4bd5-a593-bbbf659d4ba0,1.543456e+09,America/New_York,Haymarket Square,Beacon Hill,Uber,8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a,Taxi,NaN,0.91,...,0.0000,1543420800,33.85,1543399200,42.61,1543438800,30.03,1543399200,36.57,1543438800
693049,885ef635-299e-45cf-8052-c45a39e9822a,1.543456e+09,America/New_York,North End,Beacon Hill,Uber,8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a,Taxi,NaN,1.79,...,0.0000,1543420800,33.85,1543399200,42.61,1543438800,30.03,1543399200,36.57,1543438800
693060,50a7b6be-ed2a-4a11-8d36-73bd977ad66a,1.543728e+09,America/New_York,Theatre District,Haymarket Square,Uber,8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a,Taxi,NaN,1.61,...,0.0916,1543770000,36.32,1543726800,50.80,1543788000,35.84,1543748400,50.13,1543788000


In [19]:
df[df['price'].isna()].describe()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,timestamp,price,distance,latitude,longitude,temperature,apparentTemperature,precipIntensity,precipProbability,windSpeed,...,precipIntensityMax,uvIndexTime,temperatureMin,temperatureMinTime,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime
count,5.509500e+04,0.0,54532.000000,55095.000000,55095.000000,53922.000000,55095.000000,55095.000000,55095.000000,55095.000000,...,55095.000000,5.509500e+04,55095.000000,5.509500e+04,55095.000000,5.509500e+04,55095.000000,5.509500e+04,55095.000000,5.509500e+04
mean,1.544046e+09,NaN,3.198480,42.338303,-71.066230,41.385338,35.913437,0.009071,0.158384,6.317228,...,0.037436,1.544045e+09,33.476342,1.544042e+09,45.277155,1.544048e+09,29.753849,1.544049e+09,42.014321,1.544049e+09
std,6.890791e+05,NaN,16.315581,0.047654,0.020286,13.801517,7.935445,0.027140,0.350806,3.820870,...,0.055190,6.909305e+05,6.477382,6.900481e+05,5.652388,6.898592e+05,7.117307,6.870186e+05,6.951950,6.907825e+05
min,1.543204e+09,NaN,0.020000,42.214800,-71.105400,18.910000,12.130000,0.000000,0.000000,0.450000,...,0.000000,1.543162e+09,15.630000,1.543122e+09,33.510000,1.543154e+09,11.810000,1.543136e+09,28.950000,1.543187e+09
25%,1.543444e+09,NaN,1.300000,42.350300,-71.081000,36.530000,31.910000,0.000000,0.000000,3.410000,...,0.000000,1.543421e+09,30.170000,1.543399e+09,42.570000,1.543439e+09,27.760000,1.543399e+09,36.570000,1.543439e+09
50%,1.543739e+09,NaN,2.170000,42.351900,-71.063100,40.700000,35.920000,0.000000,0.000000,5.920000,...,0.000400,1.543770e+09,34.240000,1.543727e+09,44.680000,1.543788e+09,30.130000,1.543745e+09,40.950000,1.543788e+09
75%,1.544828e+09,NaN,2.860000,42.364700,-71.054200,43.910000,40.150000,0.000000,0.000000,8.410000,...,0.091600,1.544807e+09,38.940000,1.544789e+09,46.910000,1.544814e+09,35.710000,1.544789e+09,44.120000,1.544818e+09
max,1.545161e+09,NaN,697.000000,42.366100,-71.033000,171.660000,57.220000,0.144700,1.500000,75.000000,...,0.145900,1.545152e+09,43.100000,1.545192e+09,57.870000,1.545109e+09,40.050000,1.545134e+09,57.200000,1.545109e+09


### Now, we look at the rows where the price is present to contrast them with the missing-price entries.

In [9]:
df[~df["price"].isna()]

,id,timestamp,timezone,source,destination,cab_type,product_id,name,price,distance,...,precipIntensityMax,uvIndexTime,temperatureMin,temperatureMinTime,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime
0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,1.544953e+09,America/New_York,Haymarket Square,North Station,Lyft,lyft_line,Shared,5.0,0.44,...,0.1276,1544979600,39.89,1545012000,43.68,1544968800,33.73,1545012000,38.07,1544958000
1,4bd23055-6827-41c6-b23b-3c491f24e74d,1.543284e+09,America/New_York,Haymarket Square,North Station,Lyft,lyft_premier,Lux,11.0,0.44,...,0.1300,1543251600,40.49,1543233600,47.30,1543251600,36.20,1543291200,43.92,1543251600
2,981a3613-77af-4620-a42a-0c0866077d1e,1.543367e+09,America/New_York,Haymarket Square,North Station,Lyft,lyft,Lyft,7.0,0.44,...,0.1064,1543338000,35.36,1543377600,47.55,1543320000,31.04,1543377600,44.12,1543320000
3,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,1.543554e+09,America/New_York,Haymarket Square,North Station,Lyft,lyft_luxsuv,Lux Black XL,26.0,0.44,...,0.0000,1543507200,34.67,1543550400,45.03,1543510800,30.30,1543550400,38.53,1543510800
4,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,1.543463e+09,America/New_York,Haymarket Square,North Station,Lyft,lyft_plus,Lyft XL,9.0,0.44,...,0.0001,1543420800,33.10,1543402800,42.18,1543420800,29.11,1543392000,35.75,1543420800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693065,353e6566-b272-479e-a9c6-98bd6cb23f25,1.543708e+09,America/New_York,West End,North End,Uber,9a0e7b09-b92b-4c41-9779-2ad22b4d779d,WAV,9.5,1.00,...,0.0000,1543683600,31.42,1543658400,44.76,1543690800,27.77,1543658400,44.09,1543690800
693066,616d3611-1820-450a-9845-a9ff304a4842,1.543708e+09,America/New_York,West End,North End,Uber,6f72dfc5-27f1-42e8-84db-ccc7a75f6969,UberXL,13.0,1.00,...,0.0000,1543683600,31.42,1543658400,44.76,1543690800,27.77,1543658400,44.09,1543690800
693067,633a3fc3-1f86-4b9e-9d48-2b7132112341,1.543708e+09,America/New_York,West End,North End,Uber,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,9.5,1.00,...,0.0000,1543683600,31.42,1543658400,44.76,1543690800,27.77,1543658400,44.09,1543690800
693069,727e5f07-a96b-4ad1-a2c7-9abc3ad55b4e,1.543708e+09,America/New_York,West End,North End,Uber,6d318bcc-22a3-4af6-bddd-b409bfce1546,Black SUV,27.0,1.00,...,0.0000,1543683600,31.42,1543658400,44.76,1543690800,27.77,1543658400,44.09,1543690800


### Upon inspection, we observe that for all rows where the price is missing, the corresponding name column is always "Taxi". In contrast, rows with a recorded price include a variety of ride types.

In [13]:
df[df['price'].isna()]["name"].unique()

array(['Taxi'], dtype=object)

### We confirm that only "Taxi" appears in the name column when price is missing.

In [14]:
df[~df['price'].isna()]["name"].unique()

array(['Shared', 'Lux', 'Lyft', 'Lux Black XL', 'Lyft XL', 'Lux Black',
       'UberXL', 'Black', 'UberX', 'WAV', 'Black SUV', 'UberPool'],
      dtype=object)

### This shows the diversity of ride types when price is available — indicating that the missing data is specific to Taxi.
### The missing price values are not due to random data loss but are structurally missing only for Taxi rides. Since no pricing data is available for these rides, we can safely remove them from our modeling dataset.

In [24]:
df = df[~df["price"].isna()].reset_index(drop=True)

### Handling Missing Distance Values

#### The `distance` column contains approximately 1% missing values. We will impute the missing values using geographical distance between pickup and drop-off locations based on the Haversine formula.



In [36]:
df.columns

Index(['id', 'timestamp', 'timezone', 'source', 'destination', 'cab_type',
       'product_id', 'name', 'price', 'distance', 'surge_multiplier',
       'latitude', 'longitude', 'temperature', 'apparentTemperature',
       'short_summary', 'long_summary', 'precipIntensity', 'precipProbability',
       'humidity', 'windSpeed', 'windGust', 'windGustTime', 'visibility',
       'temperatureHigh', 'temperatureHighTime', 'temperatureLow',
       'temperatureLowTime', 'apparentTemperatureHigh',
       'apparentTemperatureHighTime', 'apparentTemperatureLow',
       'apparentTemperatureLowTime', 'icon', 'dewPoint', 'pressure',
       'windBearing', 'cloudCover', 'uvIndex', 'visibility.1', 'ozone',
       'sunriseTime', 'sunsetTime', 'moonPhase', 'precipIntensityMax',
       'uvIndexTime', 'temperatureMin', 'temperatureMinTime', 'temperatureMax',
       'temperatureMaxTime', 'apparentTemperatureMin',
       'apparentTemperatureMinTime', 'apparentTemperatureMax',
       'apparentTemperatureMaxTime

In [39]:
df["destination"].unique()

array(['North Station', 'Northeastern University', 'West End',
       'Haymarket Square', 'South Station', 'Fenway', 'Theatre District',
       'Beacon Hill', 'Back Bay', 'North End', 'Financial District',
       'Boston University'], dtype=object)

### Imputing Missing Distance Values Using Haversine Formula

In this dataset, the pickup location's coordinates are given in the `latitude` and `longitude` columns. However, we do not have the destination's latitude and longitude—only its name is provided.

To estimate the distance between the pickup and drop-off points for rows with missing distance values, we will follow these steps:

1. **Map Named Locations to Coordinates**:  
   Since the destination names are fixed (e.g., "Back Bay", "Beacon Hill") with only 10 unique values, we can assign approximate latitude and longitude values to each unique location in `destination` column using known coordinates for those areas.

2. **Compute Direct Distance (Haversine Distance)**:  
   Using the Haversine formula, we calculate the straight-line (great-circle) distance between the pickup coordinates and the mapped destination coordinates. This gives us the shortest possible path between two points on the Earth's surface.

3. **Adjust for Real-World Roads**:  
   In reality, passengers do not travel in a straight line, and drop-off points may vary within a neighborhood. To approximate real-world travel distance, we compute an average ratio between the actual `distance` (for rows where it's available) and the direct Haversine distance. This ratio captures the typical difference between real-world distance and direct distance.

4. **Impute Missing Values**:  
   For rows where `distance` is missing, we estimate it by multiplying the direct Haversine distance by the average ratio calculated in the previous step.

While incorporating real road distances using the Google Maps API could improve model accuracy—especially in production environments—this approach is avoided here due to its complexity, quota limitations in the free tier, and the need for external API integration.


In [ ]:
df.rename(columns={"latitude":"pickup_lat","longitude":"pickup_lon"},inplace=True)

### Assigning Coordinates Based on Location Names

We map each value in the `destination` column to its corresponding latitude and longitude using the `location_coords` dictionary.

In [ ]:
# Mapping approximate coordinates to each unique location name (based on known Boston neighborhoods)
location_coords = {
    'Back Bay': (42.3503, -71.0810),
    'Beacon Hill': (42.3588, -71.0707),
    'Financial District': (42.3555, -71.0536),
    'North End': (42.3656, -71.0543),
    'North Station': (42.3664, -71.0622),
    'South Station': (42.3522, -71.0555),
    'Theatre District': (42.3519, -71.0645),
    'West End': (42.3652, -71.0661)
}

df['dest_lat'] = df['destination'].map(lambda x: location_coords.get(x, (np.nan, np.nan))[0])
df['dest_lon'] = df['destination'].map(lambda x: location_coords.get(x, (np.nan, np.nan))[1])

### Calculating Direct Distance Using Haversine Formula

We use the Haversine formula to calculate the direct distance between pickup and drop-off coordinates. This will be stored in a new column `direct_distance_km`.

In [ ]:
from math import radians, cos, sin, asin, sqrt

def haversine(lat1, lon1, lat2, lon2):
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    # Haversine formula
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371  # Radius of Earth in kilometers
    return c * r

# Apply to all rows
df['direct_distance_km'] = df.apply(
    lambda row: haversine(row['source_lat'], row['source_lon'], row['dest_lat'], row['dest_lon']),
    axis=1
)

### Imputing Missing Values in Distance Column

To estimate the missing `distance` values, we calculate the average ratio of actual distance to direct distance (for rows where both are present). We then multiply the direct distance by this average ratio to impute the missing values.

In [40]:
# Calculate the ratio where distance is not null
valid_rows = df[~df['distance'].isna()]
avg_ratio = (valid_rows['distance'] / valid_rows['direct_distance_km']).mean()

Impute missing distances
df['distance'] = df.apply(
    lambda row: row['direct_distance_km'] * avg_ratio if pd.isna(row['distance']) else row['distance'],
    axis=1
)
